In [1]:
from __future__ import print_function, division
from builtins import range, input
# sudo pip install -U future

import numpy as np
import matplotlib.pyplot as plt

LENGTH = 3

In [57]:
class Agent:
    def __init__(self, eps=0.2, alpha=0.5):
        self.eps = eps
        self.alpha = alpha
        self.verbose = False
        self.state_history = []

    def set_value(self, value):
        self.value = value

    def set_epsilon(self, eps):
        self.eps = eps

    def set_symbol(self, symbol):
        self.sym = symbol

    def set_verbose(self, verbose):
        self.verbose = verbose

    def reset_history(self):
        self.state_history = []

    def take_action(self, env):
        # selecting action based on epsilon-greedy strategy
        r = np.random.rand()
        best_state = None
        if r < self.eps:
            if self.verbose:
                print("Taking a random action")

            possible_moves = []
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i, j))
            idx = np.random.choice(len(possible_moves))
            next_move = possible_moves[idx]
        else:
            pos2value = {}
            next_move = None
            best_value = -1
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        env.board[i, j] = self.sym
                        state = env.get_state()
                        env.board[i, j] = 0
                        pos2value[(i, j)] = self.value[state]
                        if self.value[state] > best_value:
                            best_value = self.value[state]
                            best_state = state
                            next_move = (i, j)

            if self.verbose:
                print("Taking a greedy action")
                for i in range(LENGTH):
                    for j in range(LENGTH):
#                         print(" %.3f|" % pos2value[(i, j)], end="")
                        if env.is_empty(i, j):
                            print(" %.3f|" % pos2value[(i, j)], end="")
                        else:
                            print("  ", end="")
                            if env.board[i, j] == env.x:
                                print("-1  |", end="")
                            elif env.board[i, j] == env.o:
                                print("1  |", end="")
                            else:
                                print("   |", end="")
                    print("")

        env.board[next_move[0], next_move[1]] = self.sym

    def update_state_history(self, s):
        self.state_history.append(s)

    def update(self, env):
        reward = env.reward(self.sym)
        target = reward
        for prev in reversed(self.state_history):
            value = self.value[prev] + self.alpha*(target - self.value[prev])
            self.value[prev] = value
            target = value
        self.reset_history()

In [58]:
class Human:
    def __init__(self):
        pass

    def set_symbol(self, sym):
        self.sym = sym

    def take_action(self, env):
        while True:
            move = input(
                "Enter coordinates i,j for your next move (i,j=0..2): ")
            i, j = move.split(',')
            i = int(i)
            j = int(j)
            if env.is_empty(i, j):
                env.board[i, j] = self.sym
                break

    def update(self, env):
        pass

    def update_state_history(self, s):
        pass

In [59]:
class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1 
        self.o = 1
        self.winner = None
        self.ended = False
        self.num_states = 3**(LENGTH*LENGTH)

    def is_empty(self, i, j):
        return self.board[i, j] == 0

    def reward(self, sym):
        if not self.game_over():
            return 0
        return 1 if self.winner == sym else 0

    def get_state(self):
        k = 0
        h = 0
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i, j] == 0:
                    v = 0
                elif self.board[i, j] == self.x:
                    v = 1
                elif self.board[i, j] == self.o:
                    v = 2
                h += (3**k) * v
                k += 1
        return h

    def game_over(self, force_recalculate=False):
        if not force_recalculate and self.ended:
            return self.ended

        for i in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[i].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        for j in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[:, j].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        # diagonals
        for player in (self.x, self.o):
            if self.board.trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True
            if np.fliplr(self.board).trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True

        if np.all((self.board == 0) == False):
            self.winner = None
            self.ended = True
            return True

        self.winner = None
        return False

    def is_draw(self):
        return self.ended and self.winner is None

    def draw_board(self):
        for i in range(LENGTH):
            for j in range(LENGTH):
                print("  ", end="")
                if self.board[i, j] == self.x:
                    print("x ", end="")
                elif self.board[i, j] == self.o:
                    print("o ", end="")
                else:
                    print("  ", end="")
            print("")
            
    def draw_board1(self):
        print (self.board)
        
    def draw_board2(self, p):
        print ("draw_board2 ")
        if p.verbose:
            print (self.board)
            k = self.get_state()
            print("Selecting Max value  ",p.value[k])

In [60]:
def get_state_hash_and_winner(env, i=0, j=0):
    results = []

    for v in (0, env.x, env.o):
        env.board[i, j] = v
        if j == 2:
            if i == 2:
                state = env.get_state()
                ended = env.game_over(force_recalculate=True)
                winner = env.winner
                results.append((state, winner, ended))
            else:
                results += get_state_hash_and_winner(env, i + 1, 0)
        else:
            results += get_state_hash_and_winner(env, i, j + 1)

    return results


def initialV_x(env, state_winner_triples):
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.x:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V


def initialV_o(env, state_winner_triples):
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.o:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V

In [61]:
def play_game(p1, p2, env, draw=False):

    current_player = None
    while not env.game_over():
        current_player = p2 if current_player == p1 else p1

        if draw:
            if draw == 1 and current_player == p1:
                print("Voila")
                env.draw_board1()
            if draw == 2 and current_player == p2:
                print("hi")
                env.draw_board2(p1)

        current_player.take_action(env)
        
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)

    if draw:
        env.draw_board()

    p1.update(env)
    p2.update(env)

In [62]:
if __name__ == '__main__':
    # Agent training
    p1 = Agent()
    p2 = Agent()

    env = Environment()
    state_winner_triples = get_state_hash_and_winner(env)

    Vx = initialV_x(env, state_winner_triples)
    Vo = initialV_o(env, state_winner_triples)
    p1.set_value(Vx)
    p2.set_value(Vo)

    p1.set_symbol(env.x)
    p2.set_symbol(env.o)

    print("Training ...", end='')
    for t in range(10000):
        if t % 150 == 0:
            print(".", end='')
        play_game(p1, p2, Environment())
    print("\n")

    human = Human()
    human.set_symbol(env.o)
    while True:
        p1.set_verbose(True)
        play_game(p1, human, Environment(), draw=2)
        answer = input("Play again? [Y/n]: ")
        if answer and answer.lower()[0] == 'n':
            break

Training ......................................................................

Taking a greedy action
 0.764| 0.762| 0.610|
 0.705| 0.712| 0.656|
 0.697| 0.559| 0.867|
hi
draw_board2 
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
Selecting Max value   0.867413090037
Enter coordinates i,j for your next move (i,j=0..2): 0,0
Taking a greedy action
  1  | 0.401| 0.663|
 0.335| 0.860| 0.350|
 0.520| 0.423|  -1  |
hi
draw_board2 
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Selecting Max value   0.860356487072
Enter coordinates i,j for your next move (i,j=0..2): 0,2
Taking a greedy action
  1  | 0.997|  1  |
 0.089|  -1  | 0.125|
 0.004| 0.594|  -1  |
hi
draw_board2 
[[ 1. -1.  1.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Selecting Max value   0.997248141015
Enter coordinates i,j for your next move (i,j=0..2): 2,1
Taking a greedy action
  1  |  -1  |  1  |
 0.162|  -1  | 0.589|
 0.020|  1  |  -1  |
hi
draw_board2 
[[ 1. -1.  1.]
 [ 0. -1. -1.]
 [ 0.  1. -1.]]
Selecting Max value   0.589256286621
En